In [1]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon
from folium import Marker, Popup


import requests
import io

#### Task 1: Mark all launch sites on a map

In [2]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df=pd.read_csv(url)

In [13]:
spacex_df.head(3)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356


In [3]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [4]:
# Define lists containing the launch site names and coordinates

launch_site_names = launch_sites_df["Launch Site"].tolist()
launch_site_coords = launch_sites_df[["Lat", "Long"]].values.tolist()

In [5]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)

In [107]:
for i in range(len(launch_sites_df)):
    circle = folium.Circle(launch_site_coords[i], radius=1000, color='#d35400', fill=True).add_child(folium.Popup(launch_site_names[i]))
    marker = folium.map.Marker(
        launch_site_coords[i], 
        icon=DivIcon(
            icon_size=(20,20), icon_anchor=(0,0), 
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % launch_site_names[i],
            )
        )

    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

#### Task 2: Mark the success/failed launches for each site on the map

In [7]:
# Need to create markers for all launch records
spacex_df["Color"] = spacex_df["class"].map({0: "red", 1: "green"})

In [91]:
spacex_df.head(3)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long,Color
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356,red
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356,red
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356,red


In [9]:
marker_cluster = MarkerCluster()
site_map.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
    folium.Marker(
        location=[record["Lat"], record["Long"]],
        popup=folium.Popup(f"{record['Launch Site']}", parse_html=True),
        icon=folium.Icon(color=record["Color"])
    ).add_to(marker_cluster)

site_map

#### TASK 3: Calculate the distances between a launch site to its proximities
